In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [3]:
import models
from models import get_model_type
import data
import utils
from domain import get_domain
from corpora.reference_sentence import ReferenceSentenceCorpus

In [4]:
# ls expts/rel3_tsel_ref_dial_model_separate/directions

In [5]:
# ls expts/rel3_tsel_ref_dial_model_separate/directions/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/

In [6]:
domain = get_domain('one_common')
# unk_threshold = 10
unk_threshold = 0
model_ty = get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder_3'

# model_fname = 'rel2_dial_model_separate/hierarchical_only-self_mask-pad_max-pool_rel-2_feed-context-attend_no-word-attention_ablate-properties_1_best.th'

base = 'expts/rel3_tsel_ref_dial_model_separate/directions'
dot_recurrence = 'expts/rel3_tsel_ref_dial_model_separate/dot_recurrence'
dot_recurrence_no_beliefs = 'expts/rel3_tsel_ref_dial_model_separate/dot_recurrence_no-beliefs'

model_fnames = {
    'basic': f'{base}/base_512_learned-pooling_untied_bidirectional_hidden-mention-encoder/1_ep-30.th',
    'encoder-filtered-shared': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-shared/1_ep-30.th',
    'encoder-filtered-separate': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate/1_ep-30.th',
    'encoder-filtered-separate-diffs': f'{base}/base_512_extremes_learned-pooling_untied_bidirectional_hidden-mention-encoder-filtered-separate-diffs/1_ep-30.th',
#     'dot-recurrence': f'{dot_recurrence}/base_dr-self+partner-32-mention-selection/1_ep-20.th',
    'dot-recurrence-no-beliefs': f'{dot_recurrence_no_beliefs}/base_mention-selection-ref-partner/1_ep-20.th',
    'dot-recurrence-oracle-no-beliefs': f'{dot_recurrence_no_beliefs}/base_oracle_mention-selection-ref-partner/1_ep-20.th',
#     'dot-recurrence-nm-multi': f'{dot_recurrence}/base_dr-self+partner-64-mention-selection_nmpt-multi-reference_loss-expanded/1_ep-20.th',
    'dot-recurrence-nm-multi-no-beliefs': f'{dot_recurrence_no_beliefs}/base_mention-selection-ref-partner_nmpt-multi_loss-expanded/1_ep-20.th',
}

In [7]:
seed = 1
corpus = ReferenceSentenceCorpus(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

freq cutoff: 0
vocab size: 1857
dataset data/onecommon/train_reference_1.txt, total 545056, unks 0, ratio 0.00%
dataset data/onecommon/valid_reference_1.txt, total 69348, unks 202, ratio 0.29%
dataset data/onecommon/test_reference_1.txt, total 67284, unks 210, ratio 0.31%


In [8]:
def load_json(file):
    import json
    with open(file, 'r') as f:
        return json.load(f)

In [9]:
scenarios1 = load_json('data/onecommon/aaai_train_scenarios.json')
scenarios2 = load_json('data/onecommon/aaai_train_scenarios_2.json')

In [10]:
len(scenarios1)

9000

In [11]:
len(scenarios2)

4500

In [12]:
scenarios1[0].keys()

dict_keys(['kbs', 'shared', 'uuid'])

In [13]:
train_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1
}

In [14]:
all_scenarios = {
    scenario['uuid']: scenario
    for scenario in scenarios1 + scenarios2
}

In [15]:
hit_count = 0
miss_count = 0
for scenario in scenarios2:
    uuid = scenario['uuid']
    if uuid in train_scenarios:
        assert train_scenarios[uuid] == scenario
        hit_count += 1
    else:
        miss_count += 1

In [16]:
print("hits: {}".format(hit_count))
print("misses: {}".format(miss_count))

hits: 0
misses: 4500


In [17]:
models = {
    name: utils.load_model(model_fname, map_location='cpu', prefix_dir=None)
    for name, model_fname in model_fnames.items()
}

/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.rnn_reference_model.HierarchicalRnnReferenceModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.ctx_encoder.RelationalAttentionContextEncoder3' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/dfried/miniconda3/envs/onecommon/lib/python3.6/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'models.atte

In [18]:
batchsize=1
validset, validset_stats = corpus.valid_dataset(batchsize)

100%|██████████████████████████████████████| 1038/1038 [00:03<00:00, 330.54it/s]


In [19]:
validset_stats

{'n': 74030,
 'self_n': 37015,
 'nonpadn': 69348,
 'self_nonpadn': 34674,
 'max_num_mentions': 9}

In [20]:
import torch

In [21]:
from agent import RnnAgent

In [22]:
from selfplay import make_parser

In [23]:
parser = make_parser()
args = parser.parse_args('--temperature 0.25'.split())
# args = parser.parse_args('--temperature 0.01'.split())

In [24]:
def is_selection(out):
    return '<selection>' in out

In [25]:
# display

In [26]:
from dialog import DialogLogger

In [27]:
from IPython.display import SVG, display, HTML

In [28]:
def display_svgs(svgs):
    no_wrap_div = '<div style="white-space: nowrap">' + ''.join(svgs) + '</div>'
    display(HTML(no_wrap_div))

In [29]:
SVG_SCALE=0.4

In [30]:
def display_attn(scenario, attn, agent_id, name=None):
    attn = attn.flatten().detach().numpy()
    if name is not None:
        print("{}: {}".format(name, attn))
    display_svgs([DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id], DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE)])

In [31]:
def display_attns(scenario, attns, agent_id, name=None):
    svgs = [DialogLogger._scenario_to_svg(scenario, scale=SVG_SCALE)[agent_id]]
#     if name is not None:
#         print("{}: {}".format(name, attn))
    for attn in attns:
        attn = attn.flatten().detach().numpy()
        svgs.append(DialogLogger._attention_to_svg(scenario, agent_id, attn, scale=SVG_SCALE))
    if name is not None:
        print(name)
    display_svgs(svgs)

In [32]:
# training emulation

In [33]:
trainset, trainset_stats = corpus.train_dataset(bsz=1)

100%|██████████████████████████████████████| 8304/8304 [00:33<00:00, 247.30it/s]


In [34]:
trainset[0].scenario_ids[0] in train_scenarios

True

In [35]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [36]:
for ix in range(len(trainset)):
#     ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
#     [scenario_id], real_ids, agents, chat_ids, sel_idxs, \
#     lens, rev_idxs, hid_idxs, all_num_markables = trainset[ix]
    
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = trainset[ix]

    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
#     if scenario_id == 'S_UoujG1GG6KscmnbN':
        print("found it")
        scenario = train_scenarios[scenario_id]
        break

found it


In [37]:
models.keys()

dict_keys(['basic', 'encoder-filtered-shared', 'encoder-filtered-separate', 'encoder-filtered-separate-diffs', 'dot-recurrence-no-beliefs', 'dot-recurrence-oracle-no-beliefs', 'dot-recurrence-nm-multi-no-beliefs'])

In [38]:
# model = models['encoder-filtered-separate-diffs']
# model = models['dot-recurrence']
model = models['dot-recurrence-no-beliefs']
# model = models['dot-recurrence-nm-multi-no-beliefs']

In [39]:
alice = RnnAgent(model, args)
bob = RnnAgent(model, args)

In [40]:
from models.reference_predictor import ReferencePredictor

In [41]:
from engines.beliefs import BeliefConstructor

In [42]:
from engines.rnn_reference_engine import make_dots_mentioned_multi, make_dots_mentioned_per_ref_multi

In [43]:
reference_predictor = ReferencePredictor(model.args)

In [44]:
pdb on

Automatic pdb calling has been turned ON


In [45]:
display_mentions = model.args.next_mention_prediction_type == 'multi_reference'

In [56]:
def force_contexts_and_mentions(index, datasplit=validset, candidates=20,
                                sample_temperature=0.25, argmax_temperature=0.005,
                                force_num_markables=True,
                                inference='beam'):
    ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, \
    [scenario_id], real_ids, partner_real_ids, agents, chat_ids, sel_idxs, \
    lens, rev_idxs, hid_idxs, num_markables, \
    is_self, partner_ref_inpts, partner_ref_tgts_our_view, partner_num_markables,\
    ref_disagreements, partner_ref_disagreements = datasplit[index]
    
    bsz = ctx.size(0)
    num_dots = 7
    
    # assume we've been training for a while, so that the beliefs are out of the warmup period (e.g. for models that switch from true->predicted)
    epoch = 1000
    
    dots_mentioned = make_dots_mentioned_multi(ref_tgts, model.args, bsz, num_dots)
    dots_mentioned_per_ref = make_dots_mentioned_per_ref_multi(ref_tgts, model.args, bsz, num_dots)

    partner_dots_mentioned_our_view = make_dots_mentioned_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    partner_dots_mentioned_our_view_per_ref = make_dots_mentioned_per_ref_multi(
        partner_ref_tgts_our_view, model.args, bsz, num_dots
    )
    
    belief_constructor = BeliefConstructor(
        model.args, epoch, bsz, num_dots, inpts, ref_tgts, partner_ref_tgts_our_view,
        real_ids, partner_real_ids, sel_tgt, is_self,
        partner_dots_mentioned_our_view, partner_dots_mentioned_our_view_per_ref,
        dots_mentioned, dots_mentioned_per_ref,
        ref_inpts, partner_ref_inpts,
        num_markables, partner_num_markables,
    )
    
    scenario = all_scenarios[scenario_id]
    agent = alice
    agent.feed_context(ctx.flatten(),
                       belief_constructor,
                       num_markables_to_force=num_markables[0] if force_num_markables else None)
    agent.real_ids = real_ids
    agent.agent_id = agents[0]

    writer = alice
    conv = []
    speaker = []
    words_left = 5000

    sentence_ix = 0

    assert len(inpts) == len(tgts)

    while sentence_ix < len(inpts):
        words_og = [corpus.word_dict.idx2word[ix] for ix in inpts[sentence_ix].flatten().numpy()]
        
        if ref_inpts[sentence_ix] is not None:
            dots_mentioned = (ref_tgts[sentence_ix].sum(dim=1) > 0)
            dots_mentioned_per_ref = ref_tgts[sentence_ix]
            this_num_markables = num_markables[sentence_ix]
    #         display_attn(scenario, dots_mentioned.float(), writer.agent_id, name='dots_mentioned')
            mentions = [dots_mentioned_per_ref[:,mention_ix].float()  for mention_ix in range(dots_mentioned_per_ref.size(1))]
            nm_out, _, nm_num_markables = writer.next_mention_outs[-1]
#             print('nm_num_markables: {}'.format(nm_num_markables))
#             print('num_markables: {}'.format(this_num_markables))
            if display_mentions and nm_out is not None:
#                 print(nm_out[0].size())
                _, nm_preds, _ = reference_predictor.forward(
                    True, dots_mentioned_per_ref, nm_out, nm_num_markables
                )
#                 print(nm_preds.size())
                display_attns(scenario, nm_preds.squeeze(1).float(), writer.agent_id, "to_mention_pred")
            display_attns(scenario, mentions, writer.agent_id, "to_mention_true")
#             for mention_ix in range(dots_mentioned_per_ref.size(1)):
#                 display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
        else:
            dots_mentioned = torch.zeros(1, 7).bool()
            dots_mentioned_per_ref = torch.zeros(1, 0, 7).bool()
            this_num_markables = torch.LongTensor([0])

        if is_self[sentence_ix]:
            if inference == 'sample':
                for sample_ix in range(candidates):
                    pred_outs, _, _, _, extra = writer.model.write(
                        writer.state, 
                        words_left, sample_temperature,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables
                    )
                    print('sample {}\t{}'.format(sample_ix, ' '.join(writer._decode(pred_outs, writer.model.word_dict))))
            elif inference in ['beam', 'gumbel_beam']:
                _, _, decoded = writer.model.write_beam(
                    writer.state, candidates, words_left, 
                    dots_mentioned=dots_mentioned,
                    dots_mentioned_per_ref=dots_mentioned_per_ref,
                    num_markables=this_num_markables,
                    temperature=sample_temperature if inference=='gumbel_beam' else 1.0,
                    gumbel_noise=inference=='gumbel_beam',
                )
                for cand_ix, utt in enumerate(decoded):
                    print('beam {}\t{}'.format(cand_ix, ' '.join(utt)))
            else:
                raise NotImplementedError(inference)
            pred_outs, _, _, _, extra = writer.model.write(
                writer.state, 
                words_left, argmax_temperature,
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables
            )
            print('greedy\t{}'.format(' '.join(writer._decode(pred_outs, writer.model.word_dict))))
    #         print(' '.join(words_og))

            out = writer.write(
                max_words=words_left,
                force_words=[words_og],
                start_token=words_og[0],
                dots_mentioned=dots_mentioned,
                dots_mentioned_per_ref=dots_mentioned_per_ref,
                num_markables=this_num_markables,
                ref_inpt=ref_inpts[sentence_ix],
            )
        
    #         if writer.extras[-1]['feed_ctx_attn_prob'] is not None:
    #             display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')

    #         if 'word_ctx_attn_prob_mean' in writer.extras[-1]:
    #             display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')

    #         for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
    #             display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))

        else:
            out = words_og
            writer.read(words_og,
                        dots_mentioned=dots_mentioned,
                        dots_mentioned_per_ref=dots_mentioned_per_ref,
                        num_markables=this_num_markables,
                        partner_ref_inpt=partner_ref_inpts[sentence_ix],
                        partner_num_markables=partner_num_markables[sentence_ix],
                        next_num_markables_to_force=num_markables[sentence_ix + 1] if (force_num_markables and sentence_ix < len(num_markables) - 1) else None,
                       )
        
#         writer.update_dot_h(ref_inpts[sentence_ix], partner_ref_inpts[sentence_ix],
#                             num_markables[sentence_ix], partner_num_markables[sentence_ix])

        print('human\t{}'.format(' '.join(out)))
    
        if is_self[sentence_ix]:
            if ref_inpts[sentence_ix] is not None:
                _, ref_preds, ref_stats = reference_predictor.forward(
                    ref_inpts[sentence_ix], ref_tgts[sentence_ix], writer.ref_outs[-1], this_num_markables
                )
#                 print("ref_preds size:", ref_preds.size())
                display_attns(scenario, ref_preds.squeeze(1).float(), writer.agent_id, "refs_pred")
                display_attns(scenario, ref_tgts[sentence_ix].squeeze(0).float(), writer.agent_id, "refs_true")
#                 print(ref_stats)
        else:
            if partner_ref_inpts[sentence_ix] is not None:
                _, partner_ref_preds, partner_ref_stats = reference_predictor.forward(
                    partner_ref_inpts[sentence_ix], partner_ref_tgts_our_view[sentence_ix],
                    writer.partner_ref_outs[-1], partner_num_markables[sentence_ix]
                )
    #             print(partner_ref_preds)
                display_attns(scenario, partner_ref_preds.squeeze(1).float(), writer.agent_id, "partner_refs_pred")
                display_attns(scenario, partner_ref_tgts_our_view[sentence_ix].squeeze(0).float(), writer.agent_id, "partner_refs_true")
#                 print(partner_ref_stats)

        print()
        print('-'*40)
        print()
        words_left -= len(out)
        conv.append(out)
        if is_selection(out):
            # sel_outs should be length 1
            selection_logits, _, _ = writer.sel_outs[-1]
            display_attns(scenario, selection_logits.sigmoid().float(), writer.agent_id, "sel_pred")
            sel_true = torch.zeros(1,7)
            sel_true[0,sel_tgt] = 1
            display_attns(scenario, sel_true, writer.agent_id, "sel_true")
        sentence_ix += 1

In [47]:
pdb on

Automatic pdb calling has been turned ON


In [57]:
force_contexts_and_mentions(1, inference='beam', candidates=1)

human	THEM: light grey dot close to each other , big to left and small dot to right <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

to_mention_true


beam 0	yes i see it , lets pick the larger one
greedy	yes i see it , lets pick the larger one <selection>
human	i see it . the larger to the left is about <unk> bigger in size ? <eos>
refs_pred


refs_true



----------------------------------------

human	THEM: yes , lets select larger one <selection>
partner_refs_pred


partner_refs_true



----------------------------------------

sel_pred


sel_true


In [50]:
force_contexts_and_mentions(2)

human	THEM: do you see two medium gray dots very close together where one looks to be slightly smaller than the large one ? <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

to_mention_true


beam 0	yes , i have that . let 's pick the darker one
beam 1	yes , i have that . let 's pick the larger one
beam 2	yes , i see that . let 's pick the darker one
beam 3	yes , i see that . let 's pick the larger one
beam 4	yes , i have that . let 's pick the darker one .
beam 5	yes , i have those . let 's pick the larger one
beam 6	yes , i have those . let 's pick the darker one
beam 7	yes , i see that . let 's pick the darker one .
beam 8	yes , i have that . let 's pick the larger one .
beam 9	yes , i have those . let 's pick the larger one .
beam 10	yes , i have that . let 's pick the darker one .
beam 11	yes , i see that . let 's pick the darker one .
beam 12	yes , i have those . let 's pick the darker one .
beam 13	yes , i see that . let 's pick the larger one .
beam 14	yes , i have that . let 's pick the larger one .
beam 15	yes , i have that too . let 's pick the larger one .
beam 16	yes , i have that too . let 's pick the darker one .
beam 17	yes , i see that too . let 's pick the

refs_true


...

beam 0	k click the darker one on top !
beam 1	let 's pick the darker one .
beam 2	let 's pick the darker one .
beam 3	ok let 's pick the darker one
beam 4	do you want to pick the top one ?
beam 5	do you want to pick the bottom one ?
beam 6	let 's pick the darker one on the left
beam 7	let 's pick the one on the left
beam 8	let 's pick the one on the right
beam 9	let 's pick the darker one on the right
beam 10	ok. let 's pick the darker one on the left
beam 11	do you want to pick the one on the right ?
beam 12	do you want to pick the one on the left ?
beam 13	ok. let 's pick the darker one on the right
beam 14	ok let 's pick the darker one on the left
beam 15	ok let 's pick the darker one on the right
beam 16	ok. let 's pick the darker one on the bottom
beam 17	ok. let 's choose the darker one on the left
beam 18	ok , let 's pick the darker one on the left
beam 19	ok , let 's pick the darker one on the right
greedy	ok , let 's pick the darker one on the left <selection>
human	they are b

refs_true



----------------------------------------

human	THEM: we have different views so that does n't help me but choose the larger of the two dots <selection>
partner_refs_pred


partner_refs_true



----------------------------------------



In [51]:
force_contexts_and_mentions(3, candidates=1)

human	THEM: one large black with a tiny light grey above and slightly left ? <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

to_mention_true


beam 0	yes , i have that . is there a small light grey dot to the left of the large black dot ?
greedy	yes , i have that . is there a small light grey dot to the left of the large black dot ? <eos>
human	large black dot with a tiny light dot ... yep u <unk> me to it ! <eos>
refs_pred


refs_true



----------------------------------------

human	THEM: = ) let 's choose the tiny grey . = ) <selection>
partner_refs_pred


partner_refs_true



----------------------------------------



In [52]:
force_contexts_and_mentions(4, candidates=1)

to_mention_true


beam 0	i have one small black dot in the middle of the other dots
greedy	i have one small black dot in the middle of the other dots <eos>
human	do you have a small black dot , darkest one out of all of them ? <eos>
refs_pred


refs_true



----------------------------------------

human	THEM: i have 2 small black dots . one of the black dots is south of a few light grey dots . <eos>
partner_refs_pred


partner_refs_true


...

partner_refs_true



----------------------------------------

to_mention_true


beam 0	yes i see that . let 's pick the darker one on the right
greedy	yes i see that . let 's pick the darker one on the right <selection>
human	yes , i have that , let 's choose the right dark grey one ? <eos>
refs_pred


refs_true



----------------------------------------

human	THEM: ok <selection>

----------------------------------------



In [61]:
force_contexts_and_mentions(5)

human	THEM: do you have one large dark grey dot next to a slightly smaller and slightly lighter one ? <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

to_mention_pred


to_mention_true


beam 0	do you have a large dark grey dot with a smaller black dot to the right of it and a small black dot to the right of it ?
beam 1	do you have a large dark grey dot with a small black dot to the right of it and a small black dot to the right of it ?
beam 2	do you have a large medium grey dot with a smaller black dot to the right of it and a small black dot to the right of it ?
beam 3	do you have a large dark grey dot with a smaller black dot to the right of it and a large black dot to the right of it ?
beam 4	do you have a large medium grey dot with a smaller black dot to the right of it and a large black dot to the right of it ?
beam 5	do you have a large dark grey dot with a small black dot to the right of it and a large black dot to the right of it ?
beam 6	do you have a large dark grey dot with a smaller black dot to the right of it and a smaller black dot to the right of it ?
beam 7	do you have a large dark grey dot with a small black dot to the right of it and a smaller black

...


----------------------------------------

human	THEM: yes . lets select that one <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

beam 0	ok
beam 1	okay
beam 2	ok !
beam 3	k
beam 4	okay !
beam 5	great
beam 6	ok .
beam 7	ok great
beam 8	great !
beam 9	sounds good !
beam 10	sounds good
beam 11	yes
beam 12	perfect
beam 13	alright
beam 14	ok
beam 15	ok good luck
beam 16	okay .
beam 17	ok will do
beam 18	ok sounds good
beam 19	ok , sounds good
greedy	ok <selection>
human	which ? <eos>

----------------------------------------

human	THEM: the large dark grey <selection>
partner_refs_pred


partner_refs_true



----------------------------------------



In [62]:
force_contexts_and_mentions(6)

to_mention_pred


to_mention_true


beam 0	i have two dots very close together . the one on the left is smaller and darker than the one on the right
beam 1	i have two dots very close together . the one on the right is smaller and darker than the one on the right
beam 2	i have two dots very close together . the one on the left is smaller and darker than the one on the right .
beam 3	i have two dots very close together . the one on the right is smaller and darker than the one on the right .
beam 4	i have two dots very close together . the one on the left is smaller and darker than the one on the left
beam 5	i have two dots very close together . the one on the left is smaller and darker than the one on the left .
beam 6	i have two dots very close together . the one on the right is smaller and darker than the one on the left
beam 7	i have two dots very close together . the one on the right is smaller and darker than the one on the left .
beam 8	i have two dots very close together . the one on the left is smaller and lighter 

refs_true



----------------------------------------

human	THEM: i only have two black dots one med size the other is tiny <eos>
partner_refs_pred


...


----------------------------------------

to_mention_pred


to_mention_true


beam 0	yes , let 's pick that one
beam 1	yes , let 's pick that one .
beam 2	let us select the large dark one
beam 3	let us select the large black one
beam 4	yes , let 's choose that one .
beam 5	yes . let 's pick that one .
beam 6	yes , let 's pick the large black one
beam 7	yes . let 's pick the large black one .
beam 8	yes , let 's pick the large dark one
beam 9	yes . let 's pick the large black one
beam 10	yes , let 's choose the large black one
beam 11	yes , let 's pick the large black dot
beam 12	yes , let 's pick the large black one .
beam 13	yes . let 's pick the large black dot
beam 14	yes . let 's pick the large black dot .
beam 15	yes . let 's choose the large black one .
beam 16	yes . let 's pick the large dark one .
beam 17	yes , let 's choose the large black one .
beam 18	yes . let 's choose the large black dot .
beam 19	yes , let 's pick the large dark one .
greedy	yes , let 's pick the large black one <selection>
human	okay cool me too . click that one ! <selection>
ref

refs_true



----------------------------------------



In [63]:
force_contexts_and_mentions(7)

human	THEM: i see a single light grey small dot with nothing around it really . <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

to_mention_pred


to_mention_true


beam 0	i have a small light grey dot with a large black dot to the right of it
beam 1	i have a tiny light grey dot with a large black dot to the right of it
beam 2	i have a small light grey dot with a small black dot to the right of it
beam 3	i have a small light gray dot with a large black dot to the right of it
beam 4	i have a small light grey dot with a large black dot to the left of it
beam 5	i have a small light grey dot with a large black dot above and to the right of it
beam 6	i have a tiny light grey dot with a large black dot to the left of it
beam 7	i have a small light grey dot with a small black dot to the left of it
beam 8	i have a small light grey dot with a large black dot above and to the left of it
beam 9	do you have a tiny light grey dot with a large black dot to the right of it ?
beam 10	do you have a small light grey dot with a large black dot to the right of it ?
beam 11	do you have a tiny light grey dot with a large black dot to the left of it ?
beam 12	do you hav

...


----------------------------------------

to_mention_pred


to_mention_true


beam 0	do you have a small light grey dot with a large black dot to the right of it
beam 1	do you have a small light grey dot with a large black dot to the right of it and a small black dot to the right of it ?
beam 2	do you have a small light grey dot with a large black dot to the right of it and a small black dot below it ?
beam 3	do you have a tiny light grey dot with a large black dot to the right of it and a small black dot to the right of it ?
beam 4	do you have a tiny light grey dot with a large black dot to the right of it and a small black dot below it ?
beam 5	do you have a small light grey dot with a large black dot to the right of it and a small black dot below it
beam 6	do you have a small light grey dot with a large black dot to the left of it and a small black dot to the right of it ?
beam 7	do you have a tiny light grey dot with a large black dot to the left of it and a small black dot to the right of it ?
beam 8	do you have a small light grey dot with a large black dot

refs_true



----------------------------------------

human	THEM: ok <selection>

----------------------------------------



In [64]:
force_contexts_and_mentions(8)

human	THEM: i have two larger dots . one is pale and northeast of the other dots . the other is dark and southeast of the others . <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

to_mention_pred


to_mention_true


beam 0	i do n't have that . do you have a cluster of 4 dots that make a y shape ?
beam 1	i do n't have that . do you see a cluster of 4 dots that make a y shape ?
beam 2	i do n't have that . do you have a cluster of 4 dots that form a y shape ?
beam 3	i do n't see that . do you have a cluster of 4 dots that make a y shape ?
beam 4	i do n't have that . do you see a cluster of 4 dots that form a y shape ?
beam 5	i do n't have that . do you have a cluster of 4 dots in the middle of the dots ?
beam 6	i do n't have that . do you have a cluster of 4 dots in the middle of the circle ?
beam 7	i do n't have that . i have a cluster of 4 dots that form a y with the one on the left is the lightest and smallest
beam 8	i do n't have that . i have a cluster of 4 dots that form a y with the one on the right is the lightest and smallest
beam 9	i do n't have that . i have a cluster of 4 dots that form a y with the one on the left is the lightest and the rest
beam 10	i do n't have that . i have a cluster

...


----------------------------------------

to_mention_pred


to_mention_true


beam 0	yes it is
beam 1	yes it is .
beam 2	yes it is .
beam 3	yes , let 's pick that one
beam 4	yes . let 's pick that one .
beam 5	yes , let 's pick that one
beam 6	yes , let 's choose that one
beam 7	yes . let 's choose that one .
beam 8	yes , let 's pick that one .
beam 9	yes . let 's pick that one .
beam 10	yes . let 's pick the light grey one
beam 11	yes . let 's pick the light grey one
beam 12	yes . let 's pick the light grey dot
beam 13	yes . let 's pick the light grey one .
beam 14	no . do you have a large light grey dot ?
beam 15	yes . let 's choose the light grey one .
beam 16	yes . let 's pick the light grey dot .
beam 17	no . do you have a large light grey dot at the bottom of the circle ?
beam 18	no . do you have a large light grey dot at the top of the circle ?
beam 19	no . do you have a large light gray dot at the bottom of the circle ?
greedy	yes , let 's pick the light grey one <selection>
human	yes . pick the larger one <selection>
refs_pred


refs_true



----------------------------------------



In [65]:
force_contexts_and_mentions(9)

human	THEM: i have 2 dots on the bottom , left one is darker and larger , right one is smaller and lighter <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

to_mention_pred


to_mention_true


beam 0	i do n't have that . i have a cluster of 4 dots that form a triangle with a large light grey dot in the middle of them
beam 1	i do n't have that . i have a cluster of 4 dots that form a triangle with a large light gray dot in the middle of them
beam 2	i do n't have that . i have a cluster of 4 dots that form a triangle with a large light gray dot in the middle of the circle
beam 3	i do n't have that . i have a cluster of 4 dots that form a triangle with a large light grey dot in the middle of the circle
beam 4	i do n't have that . i have a cluster of 4 dots that form a triangle with a large light gray dot in the middle of the triangle
beam 5	i do n't have that . i have a cluster of 4 dots that form a triangle with a large light grey dot in the middle of the triangle
beam 6	i do n't have that . i have a cluster of 4 dots that form a triangle with a large light gray dot in the middle of the circle .
beam 7	i do n't have that . i have a cluster of 4 dots that form a triangle with a

...


----------------------------------------

to_mention_pred


to_mention_true


beam 0	do you have a large medium gray dot with a smaller black dot to the right of it ?
beam 1	do you have a large light grey dot with a smaller black dot to the right of it ?
beam 2	do you have a large medium gray dot with a smaller black dot to the left of it ?
beam 3	do you have a large medium gray dot with a small black dot to the right of it ?
beam 4	do you have a large medium grey dot with a smaller black dot to the right of it ?
beam 5	do you have a large light grey dot with a smaller black dot to the left of it ?
beam 6	do you have a large light grey dot with a small black dot to the right of it ?
beam 7	do you have a large medium gray dot with a small black dot to the left of it ?
beam 8	do you have a large medium grey dot with a small black dot to the right of it ?
beam 9	do you have a large medium grey dot with a smaller black dot to the left of it ?
beam 10	do you have a large light grey dot with a small black dot to the left of it ?
beam 11	do you have a large medium grey

refs_true



----------------------------------------

human	THEM: yes , sure <selection>

----------------------------------------



In [66]:
force_contexts_and_mentions(10)

to_mention_pred


to_mention_true


beam 0	do you see a cluster of 4 dots with a large light grey dot in the middle ?
beam 1	do you see a cluster of 4 dots with a small light grey dot in the middle ?
beam 2	do you see a cluster of 4 dots with a small light grey dot in the middle of the circle ?
beam 3	do you see a cluster of 3 dots with a small light grey dot in the middle of the circle ?
beam 4	do you see a cluster of 4 dots with a large light grey dot in the middle of the circle ?
beam 5	do you see a cluster of 4 dots with a small light grey dot in the middle of the circle
beam 6	do you see a cluster of 4 dots with a large light grey dot in the middle of the circle
beam 7	do you see a cluster of 3 dots with a small light grey dot in the middle of the circle
beam 8	i have a cluster of 3 dots that form a triangle . the top one is the smallest and lightest
beam 9	i have a cluster of 3 dots that form a triangle . the one on the bottom is the smallest and lightest
beam 10	i have a cluster of three dots that form a triangle 

refs_true



----------------------------------------

human	THEM: yes i see that lets hit the small light gray dot <eos>
partner_refs_pred


partner_refs_true



----------------------------------------

beam 0	ok
beam 1	okay
beam 2	k
beam 3	ok !
beam 4	okay !
beam 5	great
beam 6	sounds good !
beam 7	sounds good
beam 8	cool
beam 9	yes
beam 10	ok .
beam 11	done
beam 12	k !
beam 13	alright
beam 14	great !
beam 15	perfect
beam 16	ok
beam 17	gotcha
beam 18	got it
beam 19	ok great
greedy	ok <selection>
human	k <selection>

----------------------------------------



In [67]:
force_contexts_and_mentions(11)

to_mention_pred


to_mention_true


beam 0	i have a large light grey dot to the right of a larger dark dot
beam 1	do you have a large light grey dot to the right of a larger black dot ?
beam 2	i have a large light grey dot to the right of a larger dark grey dot
beam 3	do you have a large light grey dot to the left of a larger black dot ?
beam 4	do you see a large light grey dot to the right of a larger black dot ?
beam 5	do you have a large light grey dot to the right of a larger dark dot ?
beam 6	do you have a large light grey dot to the right of a larger black dot
beam 7	do you see a large light grey dot to the right of a larger black dot
beam 8	do you have a large light grey dot to the right of a larger dark grey dot ?
beam 9	do you have a large light grey dot to the left of a larger black dot
beam 10	i have a large light grey dot with a slightly smaller dark grey dot above and to the right
beam 11	do you see a large light grey dot to the right of a larger dark grey dot ?
beam 12	i have a large light grey dot with a s

refs_true



----------------------------------------

human	THEM: i see a large black dot under an almost as big light grey dot <eos>
partner_refs_pred


...

beam 0	i see that
beam 1	i have that
beam 2	yes i see that
beam 3	i see that .
beam 4	i see that too
beam 5	yes i have that
beam 6	i have that .
beam 7	i have that too
beam 8	yes i see it
beam 9	yes , i see that
beam 10	yes , i have that
beam 11	yes , i have that .
beam 12	yes , i see that .
beam 13	yes i see that .
beam 14	i see that too .
beam 15	yes i have that .
beam 16	i think i see that
beam 17	yes , i have that too
beam 18	yes , i have that too .
beam 19	yes , i see that too .
greedy	i have that <eos>
human	great <unk> pick dots <unk> ! <eos>
refs_pred


refs_true



----------------------------------------

human	THEM: lets click the black one <selection>
partner_refs_pred


partner_refs_true



----------------------------------------



In [58]:
import json
from utils import ContextGenerator

In [43]:
ctx_gen = ContextGenerator('data/onecommon/shared_4.txt')

In [44]:
with open('data/onecommon/shared_4.json', "r") as f:
    scenario_list = json.load(f)
selfplay_scenarios = {scenario['uuid']: scenario for scenario in scenario_list}

In [45]:
scenario = None
for ix, ([scenario_id], (ctx1, ctx2), (ids1, ids2)) in enumerate(ctx_gen.iter()):
#     if scenario_id == 'S_11J8n40u4HRVduwi':
#         print('found it')
#         scenario = scenarios[scenario_id]
#         break
    if ix == 2:
        scenario = selfplay_scenarios[scenario_id]
        print(scenario_id)
        break

S_xM5bXi3lMqXHKKgP


In [46]:
import random

In [ ]:
# selfplay emulation
scenario = selfplay_scenarios[scenario_id]
for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
    agent.feed_context(ctx)
    agent.real_ids = real_ids
    agent.agent_id = agent_id
    
dot_mention_samples = 1
utt_samples = 10
    
writer, reader = [alice, bob]
conv = []
speaker = []
words_left = 5000

num_mentions = 2
num_dots_to_mention = 3

sample_temperature = 0.25
argmax_temperature = 0.005

def make_dots_mentioned_per_ref():
#     # fixed
#     return torch.BoolTensor([[1,1,0,0,0,0,0], [1,0,0,0,0,0,0]]).unsqueeze(0)
#     return torch.BoolTensor([[1,1,0,0,0,0,0]]).unsqueeze(0)

    # random initial and drop
    mentioned = torch.zeros(1,num_mentions,7).bool()
    dots_mentioned = []
    for _ in range(num_dots_to_mention):
        dot_ix = torch.randint(7, (1,))
        mentioned[:,0,dot_ix] = 1
        dots_mentioned.append(dot_ix)
        
    for mention_ix in range(1, num_mentions):
        mentioned[:,mention_ix,random.choice(dots_mentioned)] = 1
    return mentioned
    

    # random 
#     mentioned = torch.zeros(1,num_mentions,7).bool()
#     for mention_ix in range(num_mentions):
#         for _ in range(num_dots_to_mention):
#             dot_ix = torch.randint(7, (1,))
#             mentioned[:,mention_ix,dot_ix] = 1
#     return mentioned

display_context = True

while True:
    dots_mentioned_per_ref = make_dots_mentioned_per_ref()
    dots_mentioned = dots_mentioned_per_ref.max(1).values
    
    num_markables=torch.LongTensor([dots_mentioned_per_ref.size(1)])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    for us in range(utt_samples):
        outs, _, _, _, extra = writer.model.write(
            writer.ctx,
            writer.ctx_differences,
            writer.ctx_h,
            writer.reader_and_writer_lang_h,
            words_left,
            sample_temperature,
            dots_mentioned=dots_mentioned,
            dots_mentioned_per_ref=dots_mentioned_per_ref,
            num_markables=num_markables,
        )
        print('agent {}; sample {}\t{}'.format(writer.agent_id, us, ' '.join(writer._decode(outs, writer.model.word_dict))))
    print()
        
    num_markables=torch.LongTensor([num_mentions])
    if display_context:
        for mention_ix in range(dots_mentioned_per_ref.size(1)):
            display_attn(scenario, dots_mentioned_per_ref[:,mention_ix].float(), writer.agent_id, name=f'dots_mentioned_{mention_ix}')
    out = writer.write(
        max_words=words_left,
        dots_mentioned=dots_mentioned,
        dots_mentioned_per_ref=dots_mentioned_per_ref,
        num_markables=num_markables,
        temperature_override=argmax_temperature,
    )
    print('agent {}; argmax\t{}'.format(writer.agent_id, ' '.join(out)))
    print()
    
#     if display_context and writer.extras[-1].get('feed_ctx_attn_prob', None) is not None:
#         display_attn(scenario, writer.extras[-1]['feed_ctx_attn_prob'], writer.agent_id, name='feed attention')
    
#     if display_context and writer.extras[-1].get('word_ctx_attn_prob_mean', None) is not None:
#         display_attn(scenario, writer.extras[-1]['word_ctx_attn_prob_mean'], writer.agent_id, name='word attention mean')
        
#     for ix, ix_attn in enumerate(writer.extras[-1]['word_ctx_attn_probs']):
#         display_attn(scenario, ix_attn, writer.agent_id, name='t {}: {}'.format(ix, out[ix]))
        
    print('{}\t{}'.format(writer.agent_id, ' '.join(out)))
    print('-'*60)
        
    words_left -= len(out)
    conv.append(out)
    speaker.append(writer.agent_id)
    reader.read(out)
    if is_selection(out) or words_left <= 1:
        break
    writer, reader = reader, writer

NameError: name 'selfplay_scenarios' is not defined

> <ipython-input-41-e274016a0281>(2)<module>()
      1 # selfplay emulation
----> 2 scenario = selfplay_scenarios[scenario_id]
      3 for agent, agent_id, ctx, real_ids in zip([alice, bob], [0, 1], [ctx1, ctx2], [ids1, ids2]):
      4     agent.feed_context(ctx)
      5     agent.real_ids = real_ids

--KeyboardInterrupt--
